# Collecting description of offers

In [171]:
import pandas as pd
import requests
import numpy as np

In [172]:
file_name="df_commercial_rent_2022-12-24.gzip"

In [173]:
df=pd.read_parquet(file_name, columns=["hash_id"])
df.head()

,hash_id
0,1266939724
1,1242203212
2,79076188
3,88229452
4,1350403148


In [174]:
desc_indices=df.hash_id
desc_indices

0       1266939724
1       1242203212
2         79076188
3         88229452
4       1350403148
           ...    
4932    2081943900
4933    3159732572
4934    4065702236
4935    1236747100
4936    2193048412
Name: hash_id, Length: 4937, dtype: int64

In [175]:
desc_indices.drop_duplicates()
desc_indices.shape

(4937,)

In [177]:
collector_description={}

for i in range(len(desc_indices)):
    base_url = 'https://www.sreality.cz/api/cs/v2/estates/{}'.format(desc_indices[i])

    r = requests.get(base_url)

    #print("starting sleep")
    #sleep(randint(0,2))

    if r.status_code==404:
        break
    
    elif r.status_code==200:
        r_dict=r.json()
        r_dict["hash_id"]=desc_indices[i]

        collector_description[i]=r_dict
        
        print(f"Description {i+1} out of {len(desc_indices)} was scraped. hash_id: {desc_indices[i]}")

Description 1 out of 4937 was scraped. hash_id: 1266939724
Description 2 out of 4937 was scraped. hash_id: 1242203212
Description 3 out of 4937 was scraped. hash_id: 79076188
Description 4 out of 4937 was scraped. hash_id: 88229452
Description 5 out of 4937 was scraped. hash_id: 1350403148
Description 6 out of 4937 was scraped. hash_id: 2887955532
Description 7 out of 4937 was scraped. hash_id: 4263150668
Description 8 out of 4937 was scraped. hash_id: 1406854220
Description 9 out of 4937 was scraped. hash_id: 402318412
Description 11 out of 4937 was scraped. hash_id: 1318616924
Description 12 out of 4937 was scraped. hash_id: 1201089372
Description 13 out of 4937 was scraped. hash_id: 860670300
Description 14 out of 4937 was scraped. hash_id: 2119591500
Description 15 out of 4937 was scraped. hash_id: 1301042764
Description 16 out of 4937 was scraped. hash_id: 3225401164
Description 17 out of 4937 was scraped. hash_id: 507176012
Description 18 out of 4937 was scraped. hash_id: 3221151

In [178]:
items_dict={"Zlevněno":"discounted", 
"Původní cena":"price_original", 
"ID zakázky":"id_order", 
"Aktualizace":"date_update", 
"Stavba":"building_type", 
"Stav objektu":"building_condition", 
"Umístění objektu":"building_location", 
"Užitná plocha":"area_net", 
"Plocha podlahová":"area_floor", 
"Celková cena":"price_total", 
"Poznámka k ceně":"price_note", 
"Doprava":"transport", 
"Plocha zastavěná":"area_build_up", 
"Datum nastěhování":"date_moving_in", 
"Rok kolaudace":"date_building_approval", 
"Vybavení":"equipped", 
"Elektřina":"electricity", 
"Komunikace":"traffic_communication", 
"Bezbariérový":"barrier_free", 
"ID":"id", 
#"Cena":"price", 
"Energetická náročnost budovy":"energy_efficient_rating", 
"Výtah":"lift", 
"Typ domu":"building_floor_type", 
"Podlaží":"building_floor", 
"Voda":"water", 
"Plyn":"gas", 
"Odpad":"waste", 
"Topení":"heating", 
"Telekomunikace":"telecommunication", 
"Rok rekonstrukce":"date_year_reconstruction", 
"Průkaz energetické náročnosti budovy":"energy_efficient_rating_card", 
"Stav":"availability", 
"Datum zahájení prodeje":"date_offer_start", 
"Ukazatel energetické náročnosti budovy":"energy_efficient_rating_index", 
"Datum prohlídky":"date_tour", 
"Datum prohlídky do":"date_tour_to", 
"Plocha zahrady":"area_garden"}

In [179]:
description_individual = {}
r_dict_names_all=pd.Series(dtype="object")
r_dict_types_all=pd.DataFrame(columns=["name", "type"])

for page, r_dict in collector_description.items():
        
        info_relevant = pd.Series(dtype="object")
        info_relevant["description"]=r_dict["text"]["value"]

        r_dict_values=pd.DataFrame(r_dict["items"], columns =['type', 'name', 'value'])

        r_dict_names=r_dict_values["name"]
        r_dict_names_all=r_dict_names_all.append(r_dict_names[~r_dict_names.isin(r_dict_names_all)])

        r_dict_types=r_dict_values[["type", "name"]]
        r_dict_types_all=pd.concat([r_dict_types_all, r_dict_types.loc[~r_dict_types["name"].isin(r_dict_types_all["name"]),:]])

        for name_raw in items_dict.keys():

                name_clean=items_dict[name_raw]

                if name_raw not in r_dict_names.values:
                        info_relevant[name_clean]=np.nan
                elif r_dict_values[r_dict_names==name_raw]["type"].all()=="set":
                        index_nr=int(r_dict_values.index[r_dict_values['name'] == name_raw].tolist()[0])
                        info_relevant[name_clean]=r_dict_values["value"][r_dict_values["name"]==name_raw][index_nr][0]["value"].values[0]
                        del index_nr
                else:
                        info_relevant[name_clean]=r_dict_values["value"][r_dict_values["name"]==name_raw].values[0]

        #if sum(r_dict_values['name'] == "Topení")>0:
        #        index_nr=int(r_dict_values.index[r_dict_values['name'] == "Topení"].tolist()[0])
        #        r_dict_values["value"][r_dict_values["name"]=="Topení"][index_nr][0]["value"]
        #        del index_nr

        description_individual[r_dict["hash_id"]] = info_relevant

/var/folders/s_/3ph0_hm1747ch5x8tpsf9d3r0000gn/T/ipykernel_46858/2050910403.py:13: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  r_dict_names_all=r_dict_names_all.append(r_dict_names[~r_dict_names.isin(r_dict_names_all)])
/var/folders/s_/3ph0_hm1747ch5x8tpsf9d3r0000gn/T/ipykernel_46858/2050910403.py:13: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  r_dict_names_all=r_dict_names_all.append(r_dict_names[~r_dict_names.isin(r_dict_names_all)])
/var/folders/s_/3ph0_hm1747ch5x8tpsf9d3r0000gn/T/ipykernel_46858/2050910403.py:13: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  r_dict_names_all=r_dict_names_all.append(r_dict_names[~r_dict_names.isin(r_dict_names_all)])
/var/folders/s_/3ph0_hm1747ch5x8tpsf9d3r0000gn/T/ipykernel_46858

In [180]:
r_dict_types_all

,name,type
0,Celková cena,price_czk
1,Cena za m²,price
2,Poznámka k ceně,string
3,ID zakázky,string
4,Aktualizace,edited
5,Stavba,string
6,Stav objektu,string
7,Typ domu,string
8,Podlaží,string
9,Užitná plocha,area


In [181]:
#r_dict_types_all
#boolean_formats=["boolean"]
#float_formats=["integer", "price_czk", "price_czk_old", "area", ""]

In [182]:
print("Add these values to your dictionary:")
print(r_dict_names_all[~r_dict_names_all.isin(items_dict.keys())])

Add these values to your dictionary:
1                  Cena za m²
10                      Garáž
0                        Cena
10                  Parkování
11               Výška stropu
10            Počet kanceláří
9     Datum ukončení výstavby
dtype: object


In [183]:
df_desc = pd.concat(description_individual).unstack()
df_desc.head()

,description,discounted,price_original,id_order,date_update,building_type,building_condition,building_location,area_net,area_floor,...,heating,telecommunication,date_year_reconstruction,energy_efficient_rating_card,availability,date_offer_start,energy_efficient_rating_index,date_tour,date_tour_to,area_garden
1266939724,K okamžitému pronájmu je připravena světlá a p...,NaN,NaN,228592,24.12.2022,Cihlová,Velmi dobrý,NaN,52,NaN,...,"[{'name': 'Topení', 'value': 'Lokální plynové'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1242203212,Nově zrekonstruované studio se nachází v činžo...,NaN,NaN,128916-1,27.12.2022,Cihlová,Po rekonstrukci,NaN,31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79076188,Nabízíme k pronájmu obchodní prostor o výměře ...,NaN,NaN,VITA1.102Pr,Dnes,Cihlová,Novostavba,NaN,412,412,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88229452,"Pronájem kanceláře o výměře 34,8 m2 na dobu ur...",NaN,NaN,RAD_A3.107,Dnes,Skeletová,Dobrý,NaN,40,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1350403148,Pronájem obchodního prostoru o rozloze 174 m2 ...,NaN,NaN,ANK0063-0002,26.12.2022,Smíšená,Novostavba,NaN,200,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
df_desc.energy_efficient_rating_index.unique()

array([nan, '142,0', '2,0', '418,3', '122,0', '108,0', '154,0', '77,0',
       '164,0', '235,0', '85,0', '97,0', '58,0', '845,849', '132,0',
       '304,0', '167,0', '1,0', '151,0', '155,0', '112,0', '264,0',
       '416100,0', '196,0', '109,0', '107,0', '123,0', '86,0', '45,0',
       '70,0', '66,0', '203,0', '157,0', '75,0', '180,0', '146,0', '98,0',
       '136,0', '126,0', '119,0', '174,0', '1212,4', '172,0', '80,0',
       '138,0', '80,6', '217,0', '102,0', '1302,0', '800,0', '197,0',
       '752,9', '124,0', '269,0', '60,0', '50,0', '175,0', '121,0',
       '192,0', '118,0', '148,0', '2001,56', '90,0', '106,0', '232,0',
       '159,0', '152,0', '79,0', '143,0', '369,0', '81,0', '120,0',
       '147,0', '153,0', '94,0', '95,0', '25,0', '500,0', '250,9', '96,0',
       '99,0', '12,0', '238,0', '59,0', '175,5', '252,0', '602,73',
       '98,2', '87,0', '350,0', '249,0', '365,0', '250,0'], dtype=object)

In [185]:
df_desc["equipped"]=df_desc["equipped"].map({True: "ano", False: "ne", "Částečně":"částečně"}) 

### Saving data

In [186]:
df_desc.reset_index(inplace=True)
df_desc = df_desc.rename(columns = {'index':'hash_id'})
df_desc.to_parquet((file_name[:-5]+"_desc.gzip")) 

In [187]:
# end of final code

# New version of scraping - proxies and multiple requests at the same time

In [188]:
from fp.fp import FreeProxy
import gevent
from gevent import monkey, pool
#https://stackoverflow.com/questions/15322701/gevent-pool-with-nested-web-requests
#https://pypi.org/project/free-proxy/

In [189]:
proxy = FreeProxy(country_id=['DE', "CZ", "BR"]).get()

In [190]:
urls=np.add("https://www.sreality.cz/api/cs/v2/estates/", desc_indices)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U42'), dtype('int64')) -> None

In [ ]:
monkey.patch_all()

jobs = []
links = []
p = pool.Pool(10)

add

urls = [
    'http://www.google.com', 
    # ... another 100 urls
]
    
def get_jsons(url):
    r = requests.get(url)
    if r.status_code == 200:
        r_dict = r.json()
        r_dict["hash_id"]=desc_indices[i]

        collector_description[i]=r_dict
        
for url in urls:
    jobs.append(p.spawn(get_jsons, url))
gevent.joinall(jobs)